In [2]:
import os
import weaviate

client = weaviate.CollectionClient(
    url="http://localhost:8080",
    grpc_port=50051,
)

In [3]:
client.get_meta()

{'hostname': 'http://[::]:8080',
 'modules': {'generative-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'Generative Search - OpenAI'},
  'reranker-cohere': {'documentationHref': 'https://txt.cohere.com/rerank/',
   'name': 'Reranker - Cohere'},
  'text2vec-cohere': {'documentationHref': 'https://docs.cohere.ai/embedding-wiki/',
   'name': 'Cohere Module'},
  'text2vec-huggingface': {'documentationHref': 'https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task',
   'name': 'Hugging Face Module'},
  'text2vec-openai': {'documentationHref': 'https://platform.openai.com/docs/guides/embeddings/what-are-embeddings',
   'name': 'OpenAI Module'}},
 'version': '1.21.0'}

In [3]:
client.schema.delete_class("Question")

In [4]:
import weaviate.collection.classes as wvc

questions = client.collection.create(
    wvc.CollectionConfig(
        name="Question",
        vectorizer=wvc.Vectorizer.TEXT2VEC_OPENAI,
        properties=[
            wvc.Property(name="answer", data_type=wvc.DataType.TEXT),
            wvc.Property(name="question", data_type=wvc.DataType.TEXT),
        ],
    )
)

In [5]:
import requests
import json
resp = requests.get('https://raw.githubusercontent.com/weaviate-tutorials/quickstart/main/data/jeopardy_tiny.json')
data = json.loads(resp.text)  # Load data

In [6]:
wv_objects = list()
for i, d in enumerate(data):  # Batch import data
    properties = {
        "answer": d["Answer"],
        "question": d["Question"],
        "category": d["Category"],
    }
    if i == 5:
        properties["answer"] = [42]
    wv_obj = wvc.DataObject(
        data=properties,
    )
    wv_objects.append(wv_obj)

In [7]:
response = questions.data.insert_many(wv_objects)
print(response)

_BatchReturn(all_responses=[UUID('6a00b418-7de4-46dc-8703-39f60f89d864'), UUID('9c6d70b6-261a-4794-854e-7514602aabc6'), UUID('7b8d87a9-0ed9-4861-8fe1-8e6855f60619'), UUID('29f2630d-fad8-469e-a1eb-ff1eb6fe6c78'), UUID('1593c6b3-1d45-42b7-a6cc-39f857fb3327'), Error(message="invalid text property 'answer' on class 'Question': not a string, but []interface {}", code=None, original_uuid=None), UUID('076a8e06-6918-4986-ab81-82927ea4d92c'), UUID('b74efeba-96bb-43d1-8e4e-2754fdfc568e'), UUID('91a0f044-7bb6-46e5-9dea-90b4b43569b7'), UUID('1170c856-30ee-469e-85cd-66b823bf759b')], uuids={0: UUID('6a00b418-7de4-46dc-8703-39f60f89d864'), 1: UUID('9c6d70b6-261a-4794-854e-7514602aabc6'), 2: UUID('7b8d87a9-0ed9-4861-8fe1-8e6855f60619'), 3: UUID('29f2630d-fad8-469e-a1eb-ff1eb6fe6c78'), 4: UUID('1593c6b3-1d45-42b7-a6cc-39f857fb3327'), 6: UUID('076a8e06-6918-4986-ab81-82927ea4d92c'), 7: UUID('b74efeba-96bb-43d1-8e4e-2754fdfc568e'), 8: UUID('91a0f044-7bb6-46e5-9dea-90b4b43569b7'), 9: UUID('1170c856-30ee-4

In [8]:
response.has_errors

True

In [9]:
response.errors

{5: Error(message="invalid text property 'answer' on class 'Question': not a string, but []interface {}", code=None, original_uuid=None)}

In [23]:
for k, v in response.errors.items():
    print(wv_objects[k])
    print(v)

DataObject(data={'answer': [42], 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'}, uuid=None, vector=None)
Error(message="invalid text property 'answer' on class 'Question': not a string, but []interface {}", code=None, original_uuid=None)


In [20]:
print(wv_objects[k])

DataObject(data={'answer': [42], 'question': "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification", 'category': 'SCIENCE'}, uuid=None, vector=None)